<a href="https://colab.research.google.com/github/rumman-adnan/Assignment-Sensors-Data/blob/main/sensors_data_analytics_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score



In [ ]:
# read data
# currently dont use this
train_data = pd.read_csv('/content/drive/MyDrive/Python work/sim_data/CSV data/training_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Python work/sim_data/CSV data/testing_data.csv')
print(train_data.shape)
print(test_data.shape)


(20000, 1002)
(10000, 1002)


In [ ]:
train_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_20000_0.1_90_140_train.npy')
test_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_10000_0.1_141_178_test.npy')

print("The shape of trained data is: ",train_data.shape)
print("The shape of tested data is: ",test_data.shape)

The shape of trained data is:  (20000, 1006)
The shape of tested data is:  (10000, 1006)


In [ ]:
# Extract features and target variables from the training dataset
train_sensor_data = train_data[:, :1000]
train_ID = train_data[:, 1000]
train_time = train_data[:, 1001]
train_H = train_data[:, 1002]
train_R = train_data[:, 1003]
train_S = train_data[:, 1004]
train_D = train_data[:, 1005]

# Extract features and target variables from the test dataset
test_sensor_data = test_data[:, :1000]
test_ID = test_data[:, 1000]
test_time = test_data[:, 1001]
test_H = test_data[:, 1002]
test_R = test_data[:, 1003]
test_S = test_data[:, 1004]
test_D = test_data[:, 1005]

# Display the shape of the extracted arrays to confirm extraction
train_sensor_data.shape, train_S.shape, train_D.shape, test_sensor_data.shape, test_S.shape, test_D.shape


((20000, 1000), (20000,), (20000,), (10000, 1000), (10000,), (10000,))

In [ ]:
from sklearn.preprocessing import StandardScaler

# Reshape the data to fit the input requirements of a 1D CNN model
# The shape should be (num_samples, num_time_steps, num_features)
train_sensor_data_cnn = train_sensor_data.reshape(train_sensor_data.shape[0], train_sensor_data.shape[1], 1)
test_sensor_data_cnn = test_sensor_data.reshape(test_sensor_data.shape[0], test_sensor_data.shape[1], 1)

# Normalize the features
scaler = StandardScaler()
train_sensor_data_cnn = scaler.fit_transform(train_sensor_data_cnn.reshape(-1, 1)).reshape(train_sensor_data_cnn.shape)
test_sensor_data_cnn = scaler.transform(test_sensor_data_cnn.reshape(-1, 1)).reshape(test_sensor_data_cnn.shape)

# Show the shape of the reshaped and normalized data
train_sensor_data_cnn.shape, test_sensor_data_cnn.shape


((20000, 1000, 1), (10000, 1000, 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the CNN model architecture
def build_cnn_model(input_shape):
    model = Sequential()

    # Convolutional layers
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2))  # Two output nodes to predict both systolic and diastolic BP

    # Compile the model
    # model.compile(optimizer=Adam(), loss='mse')
    model.compile(optimizer='adam', loss='mse', epochs = 50, verbose = True,
              metrics=['accuracy'])

    return model

# Build the CNN model
cnn_model = build_cnn_model((train_sensor_data_cnn.shape[1], 1))
model.fit(train_sensor_data_cnn.shape[1], epochs=2, steps_per_epoch=10)
# Display the model architecture
cnn_model.summary()


In [ ]:
from sklearn.metrics import mean_absolute_error

# Make predictions on the test set
predictions = cnn_model.predict(test_sensor_data_cnn)

# Separate the predictions for systolic and diastolic BP
pred_S, pred_D = predictions[:, 0], predictions[:, 1]

# Compute the MAE for systolic and diastolic BP
mae_S = mean_absolute_error(test_S, pred_S)
mae_D = mean_absolute_error(test_D, pred_D)

print(f"Mean Absolute Error for Systolic BP: {mae_S}")
print(f"Mean Absolute Error for Diastolic BP: {mae_D}")


313/313 [==============================] - 2s 5ms/step
Mean Absolute Error for Systolic BP: 159.4602992392715
Mean Absolute Error for Diastolic BP: 79.91103181238296


### Apply CNN with Numpy arrays
- Read numpy arrays instead of CSV data and apply algorithms on it
- large epochs and paramters tunning till we acheive MAE 3


In [8]:
# Load the numpy files
test_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_10000_0.1_141_178_test.npy')
train_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_20000_0.1_90_140_train.npy')

# Get the shape of the data arrays
test_shape = test_data.shape
train_shape = train_data.shape

test_shape, train_shape, test_data[:3], train_data[:3]


((10000, 1006),
 (20000, 1006),
 array([[-7.14960675e-08, -3.61296976e-08,  3.41059619e-07, ...,
          1.10000000e+01,  1.72000000e+02,  8.30000000e+01],
        [ 2.77480001e-08,  2.82283796e-08,  2.99275705e-08, ...,
          1.30000000e+01,  1.73000000e+02,  7.90000000e+01],
        [ 1.00426931e-07,  1.00875737e-07,  3.30280515e-07, ...,
          2.10000000e+01,  1.46000000e+02,  7.30000000e+01]]),
 array([[-2.45845714e-07, -2.06162897e-07,  1.56348382e-06, ...,
          1.90000000e+01,  9.10000000e+01,  9.50000000e+01],
        [ 2.70396897e-07,  4.12740460e-07,  2.72092277e-06, ...,
          1.60000000e+01,  1.01000000e+02,  6.10000000e+01],
        [ 1.97552550e-07,  5.92946672e-07,  4.53845770e-06, ...,
          1.00000000e+01,  1.31000000e+02,  8.10000000e+01]]))

In [9]:
# Splitting the data into features and targets

# Extracting sensor data (features)
X_train = train_data[:, :1000]
X_test = test_data[:, :1000]

# Extracting target variables for Systolic (S) and Diastolic (D) blood pressure only
Y_train_SD = train_data[:, -2:]
Y_test_SD = test_data[:, -2:]

# Names of the target variables for reference
target_names_SD = ["Systolic BP", "Diastolic BP"]

Y_train_SD.shape, Y_test_SD.shape


((20000, 2), (10000, 2))

In [ ]:
# Simple CNN,

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import mean_absolute_error

# Reshaping data to be suitable for CNN (batch_size, steps, input_dim)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Model Creation
model = Sequential()

# Adding Convolutional layers
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten and add Dense layers
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(2))  # Two output units for S and D

# Compiling the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=MeanSquaredError())

# Train the model over several epochs
epochs = 10
history = model.fit(X_train_reshaped, Y_train_SD, validation_split=0.2, epochs=epochs, batch_size=32)

# Evaluate the model's performance on the test set
test_loss = model.evaluate(X_test_reshaped, Y_test_SD)

# Predicting S and D from the test set
predictions = model.predict(X_test_reshaped)

# Calculating the MAE score
mae_scores = mean_absolute_error(Y_test_SD, predictions, multioutput='raw_values')

mae_scores


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.optimizers import Adam

# Reshaping data to be suitable for CNN (batch_size, steps, input_dim)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Model Creation
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(2))

# Define optimizer and loss
optimizer = Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()

# Custom training loop
epochs = 500  # Maximum epochs
batch_size = 32
desired_mae = 3
steps_per_epoch = len(X_train_reshaped) // batch_size

for epoch in range(epochs):
    print("\nEpoch {}/{}".format(epoch+1, epochs))

    for step in range(steps_per_epoch):
        # Create a batch of data
        X_batch = X_train_reshaped[step*batch_size:(step+1)*batch_size]
        Y_batch = Y_train_SD[step*batch_size:(step+1)*batch_size]

        # Forward pass
        with tf.GradientTape() as tape:
            predictions = model(X_batch, training=True)
            loss_value = loss_fn(Y_batch, predictions)

        # Backward pass
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Evaluate after each epoch
    predictions = model(X_test_reshaped)
    mae_scores = mean_absolute_error(Y_test_SD, predictions, multioutput='raw_values')
    print("MAE after epoch {}: {}".format(epoch+1, mae_scores))

    # If desired MAE is achieved, stop training
    if mae_scores[0] <= desired_mae and mae_scores[1] <= desired_mae:
        print("Desired MAE achieved. Stopping training.")
        break



Epoch 1/500


MAE after epoch 1: [41.630964   10.49081807]

Epoch 2/500
MAE after epoch 2: [43.12377963 10.26475824]

Epoch 3/500
MAE after epoch 3: [44.26942156 10.22227547]

Epoch 4/500
MAE after epoch 4: [44.59012236 10.22488003]

Epoch 5/500
MAE after epoch 5: [44.25564566 10.26384892]

Epoch 6/500
MAE after epoch 6: [43.75580719 10.34435228]

Epoch 7/500
MAE after epoch 7: [43.55166467 10.42435823]

Epoch 8/500
MAE after epoch 8: [42.4928696  10.61349678]

Epoch 9/500
MAE after epoch 9: [43.090744   10.56772761]

Epoch 10/500
MAE after epoch 10: [45.12046623 10.34711438]

Epoch 11/500
MAE after epoch 11: [45.65129122 10.30230228]

Epoch 12/500
MAE after epoch 12: [45.56038377 10.30603502]

Epoch 13/500
MAE after epoch 13: [45.39397685 10.31307828]

Epoch 14/500
MAE after epoch 14: [45.27833308 10.31708251]

Epoch 15/500
MAE after epoch 15: [45.2004636  10.31616693]

Epoch 16/500
MAE after epoch 16: [45.14730025 10.31238877]

Epoch 17/500
MAE after epoch 17: [45.111418   10.30904246]

Epoch 18/5

KeyboardInterrupt: ignored

** Apply LST

** LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error

# Since LSTM expects input in the shape (batch_size, timesteps, features), our data is already suitable
X_train_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Model Creation
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], 1)))
model.add(Dense(50, activation='relu'))
model.add(Dense(2))

# Define optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()

# Custom training loop
epochs = 500  # Maximum epochs
batch_size = 32
desired_mae = 3
steps_per_epoch = len(X_train_lstm) // batch_size

for epoch in range(epochs):
    print("\nEpoch {}/{}".format(epoch+1, epochs))

    for step in range(steps_per_epoch):
        # Create a batch of data
        X_batch = X_train_lstm[step*batch_size:(step+1)*batch_size]
        Y_batch = Y_train_SD[step*batch_size:(step+1)*batch_size]

        # Forward pass
        with tf.GradientTape() as tape:
            predictions = model(X_batch, training=True)
            loss_value = loss_fn(Y_batch, predictions)

        # Backward pass
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Evaluate after each epoch
    predictions = model(X_test_lstm)
    mae_scores = mean_absolute_error(Y_test_SD, predictions, multioutput='raw_values')
    print("MAE after epoch {}: {}".format(epoch+1, mae_scores))

    # If desired MAE is achieved, stop training
    if mae_scores[0] <= desired_mae and mae_scores[1] <= desired_mae:
        print("Desired MAE achieved. Stopping training.")
        break


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error

# Data shape is already suitable for LSTM: (batch_size, timesteps, features)
X_train_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Model Creation
model = Sequential()

# Adding LSTM layer optimized for cuDNN
model.add(LSTM(50, activation='tanh', recurrent_activation='sigmoid', input_shape=(X_train_lstm.shape[1], 1)))

# Dense layers for regression
model.add(Dense(50, activation='relu'))
model.add(Dense(2))  # Two output units for S and D

# Define optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()

# Custom training loop
epochs = 500  # Maximum epochs
batch_size = 32
desired_mae = 3
steps_per_epoch = len(X_train_lstm) // batch_size

for epoch in range(epochs):
    print("\nEpoch {}/{}".format(epoch+1, epochs))

    for step in range(steps_per_epoch):
        # Create a batch of data
        X_batch = X_train_lstm[step*batch_size:(step+1)*batch_size]
        Y_batch = Y_train_SD[step*batch_size:(step+1)*batch_size]

        # Forward pass
        with tf.GradientTape() as tape:
            predictions = model(X_batch, training=True)
            loss_value = loss_fn(Y_batch, predictions)

        # Backward pass
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Evaluate after each epoch
    predictions = model(X_test_lstm)
    mae_scores = mean_absolute_error(Y_test_SD, predictions, multioutput='raw_values')
    print("MAE after epoch {}: {}".format(epoch+1, mae_scores))

    # If desired MAE is achieved, stop training
    if mae_scores[0] <= desired_mae and mae_scores[1] <= desired_mae:
        print("Desired MAE achieved. Stopping training.")
        break


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Model Creation
model = Sequential()
model.add(LSTM(10, activation='tanh', recurrent_activation='sigmoid', input_shape=(X_train_lstm.shape[1], 1)))
model.add(Dense(2))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error')

# A brief summary
model.summary()

# Test with a small batch
X_sample = X_train_lstm[:32]
Y_sample = Y_train_SD[:32]
model.predict(X_sample)
